# Linear Filter Improvements

The class **`nengolib.LinearFilter()`** is an improved implementation of, and a drop-in replacement for, **`nengo.LinearFilter()`**. This `Synapse` object is much more efficient when the order of the transfer function is greater than 2.

In [ ]:
import numpy as np

import nengo
import nengolib

def make_runner(synapse, n_neurons, dims, t):
    with nengolib.Network() as model:
        stim = nengo.Node(output=lambda t: np.sin(t*np.ones(dims)))
        x = nengo.Ensemble(n_neurons, dims)
        nengo.Connection(stim, x, synapse=synapse)
        sim = nengo.Simulator(model)
        return lambda: sim.run(t, progress_bar=False)

The following test compares both implementations when connecting a node to a 2D-ensemble of 100 neurons using a 10th order synapse. We see that the overall runtime of a simulation can be significantly reduced with this change in synapse object.

In [ ]:
from timeit import Timer

order = 10
n_neurons = 100
dims = 2
t = 0.05
trials = 500

for module in (nengo, nengolib):
    x = np.zeros(order)
    x[0] = 1.0
    synapse = module.LinearFilter(x, x)
    stmt = make_runner(synapse, n_neurons, dims, t)
    print module.__name__, Timer(stmt).timeit(trials)